## Lab 7: Eel Distribution Modeling with XGBoost

Author: Eva Newby

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

### Step 1: Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# separate target and features
y = model_data['Angaus'] # target variable
X = model_data.loc[:, model_data.columns != 'Angaus'] # features

# View column names
print(model_data.info())

# Print all column names
print(model_data.columns)

# Print categorical column
print("Categorical columns:", model_data.select_dtypes(include=['object']).columns)

# Encode categorical features
cat_feat = 'Method'
le = LabelEncoder()
X['Method'] = le.fit_transform(X['Method']) # encoding x

# split data into testing and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 808)

X_train.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Angaus      1000 non-null   int64  
 1   SegSumT     1000 non-null   float64
 2   SegTSeas    1000 non-null   float64
 3   SegLowFlow  1000 non-null   float64
 4   DSDist      1000 non-null   float64
 5   DSMaxSlope  1000 non-null   float64
 6   USAvgT      1000 non-null   float64
 7   USRainDays  1000 non-null   float64
 8   USSlope     1000 non-null   float64
 9   USNative    1000 non-null   float64
 10  DSDam       1000 non-null   int64  
 11  Method      1000 non-null   object 
 12  LocSed      824 non-null    float64
dtypes: float64(10), int64(2), object(1)
memory usage: 101.7+ KB
None
Index(['Angaus', 'SegSumT', 'SegTSeas', 'SegLowFlow', 'DSDist', 'DSMaxSlope',
       'USAvgT', 'USRainDays', 'USSlope', 'USNative', 'DSDam', 'Method',
       'LocSed'],
      dtype='object')
Categorical column

/tmp/ipykernel_580851/4021797446.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Method'] = le.fit_transform(X['Method']) # encoding x


SegSumT       float64
SegTSeas      float64
SegLowFlow    float64
DSDist        float64
DSMaxSlope    float64
USAvgT        float64
USRainDays    float64
USSlope       float64
USNative      float64
DSDam           int64
Method          int64
LocSed        float64
dtype: object

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [3]:
# Create baseline model
baseline = XGBClassifier(n_estimators=1000,
                         learning_rate=0.1,
                         eval_metric="logloss",
                         early_stopping_rounds=50,
                         random_state = 808)

# Fit baseline model
baseline.fit(X_train, y_train,
            eval_set = [(X_val,  y_val)],
            verbose = True)

# Save best iterations
best_n_est = baseline.best_iteration

# Find and print the best number of trees from fitted baseline model
print("Best number of trees:", baseline.best_iteration)

[0]	validation_0-logloss:0.51354
[1]	validation_0-logloss:0.48803
[2]	validation_0-logloss:0.46738
[3]	validation_0-logloss:0.45126
[4]	validation_0-logloss:0.43766
[5]	validation_0-logloss:0.42555
[6]	validation_0-logloss:0.41728
[7]	validation_0-logloss:0.40769
[8]	validation_0-logloss:0.40082
[9]	validation_0-logloss:0.39562
[10]	validation_0-logloss:0.39263
[11]	validation_0-logloss:0.38984
[12]	validation_0-logloss:0.38489
[13]	validation_0-logloss:0.38327
[14]	validation_0-logloss:0.37803
[15]	validation_0-logloss:0.37652
[16]	validation_0-logloss:0.37337
[17]	validation_0-logloss:0.37156
[18]	validation_0-logloss:0.36987
[19]	validation_0-logloss:0.36868
[20]	validation_0-logloss:0.36809
[21]	validation_0-logloss:0.36721
[22]	validation_0-logloss:0.36763
[23]	validation_0-logloss:0.36570
[24]	validation_0-logloss:0.36577
[25]	validation_0-logloss:0.36666
[26]	validation_0-logloss:0.36559
[27]	validation_0-logloss:0.36552
[28]	validation_0-logloss:0.36422
[29]	validation_0-loglos

### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [4]:
# build parameter distribution
param_dist_learning = {'learning_rate': uniform(loc = 0.01, scale = 0.3)}

# Define new XGB classifier
tuning_classifier = XGBClassifier(n_estimators = best_n_est,
                                 random_state = 808)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(estimator = tuning_classifier,
                                   param_distributions = param_dist_learning,
                                   n_iter = 20,
                                   scoring = 'roc_auc', 
                                   cv = 3,
                                   random_state=808,
                                   verbose=1)

# Fit the randomized search using the training data.
random_search.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  verbose=False)

# save learning rate
best_learning = random_search.best_params_['learning_rate']

# Print the best learning rate found.
print("Best learning rate:", random_search.best_params_['learning_rate']) 

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best learning rate: 0.263902043687467


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [5]:
#  Initialize model with the best number of trees and learning rate
optimal_tree_model = xgb.XGBClassifier(n_estimators = best_n_est,
                                       learning_rate = best_learning,
                                       eval_metric = "logloss",
                                       random_state = 808)

# define a parameter dictionary
parameters_tree  = {'max_depth': randint(3,11),
              'min_child_weight': randint(1, 11),
              'gamma': uniform(0.05, 0.05)}


In [6]:
# find best combo
random_search_tree = RandomizedSearchCV(estimator = optimal_tree_model,
                                   param_distributions = parameters_tree,
                                   n_iter = 20,
                                   cv = 5,
                                   random_state = 808,
                                   verbose = 1)

# Fit to training data
random_search_tree.fit(X_train, y_train)


# Save best tree params
best_tree_params = random_search_tree.best_params_

# Print the best parameters
print(f"The best parameters are: {random_search_tree.best_params_}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
The best parameters are: {'gamma': 0.09077113838342292, 'max_depth': 6, 'min_child_weight': 9}


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [7]:
# Add values in from previous steps  
best_tree_parameters = {'max_depth': 5,
                        'min_child_weight': 8,
                        'gamma': 0.057515662635219456}

# Initialize Model for Tuning
stochastic_tuning_model = xgb.XGBClassifier(n_estimators = best_n_est,
                            learning_rate = best_learning,
                             **best_tree_parameters,
                            random_state = 808)

# Parameter dictionary
param_dict = {'subsample': uniform(loc=0.5, scale=0.5),
              'colsample_bytree': uniform(loc=0.5, scale=0.5),
              'random_state': [808]}

# Initialize randomized search
random_search_stochastic_tuning = RandomizedSearchCV(estimator = stochastic_tuning_model,
                                         param_distributions = param_dict)

# Fit model
random_search_stochastic_tuning.fit(X_train, y_train)

# Save best stochastic params
best_stochastic_params = random_search_stochastic_tuning.best_params_

# Print best parameters
print(f"Best stochastic parameters: {random_search_stochastic_tuning.best_params_}")

Best stochastic parameters: {'colsample_bytree': 0.8691044685768827, 'random_state': 808, 'subsample': 0.8967409627582829}


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [15]:
# Prepare evaluation data
y_eval = eval_data['Angaus_obs'] # target variable
X_eval = eval_data.loc[:, model_data.columns != 'Angaus_obs'] # features

#Prepare the evaluation data
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size = 0.2, random_state = 808)

# Initialize the StandardScaler to scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Fit on the training data
X_eval = scaler.transform(X_eval)        # Transform the evaluation data 

# Train final model using best parameters
final_model = xgb.XGBClassifier(eval_metric="logloss",
                            #random_state=808,
                            **best_tree_params, 
                            **best_stochastic_params)

# fit final model to full training dataset
final_model.fit(X_train, y_train)

# predict on evaluation data
y_pred = final_model.predict(X_eval)
y_pred_proba = final_model.predict_proba(X_eval)

# Calculate accuracy
accuracy = accuracy_score(y_eval, y_pred)

# Print results
print(f"Accuracy on Evaluation Data: {accuracy:.3f}")

Accuracy on Evaluation Data: 0.840


### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [19]:
# Compute AUC
y_pred_proba = final_model.predict_proba(X_eval)[:, 1]
auc = roc_auc_score(y_eval, y_pred_proba)

# Compute feature importances
fimportances = final_model.feature_importances_

# Print both
print(f"AUC: {auc:.3f}")
print("Feature Importances:")
for feature, importance in zip(X.columns, fimportances):  
    print(f"{feature}: {importance:.3f}")

AUC: 0.897
Feature Importances:
SegSumT: 0.190
SegTSeas: 0.062
SegLowFlow: 0.043
DSDist: 0.058
DSMaxSlope: 0.128
USAvgT: 0.060
USRainDays: 0.082
USSlope: 0.066
USNative: 0.108
DSDam: 0.000
Method: 0.148
LocSed: 0.056


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

AUC from Elith et al: 0.869